In [ ]:
#Required only when there are SubModel(s)
import sys 
sys.path.append(r"C:\Users\endofs\AppData\Local\Temp\ArcGISProTemp4060")
# -*- coding: utf-8 -*-
"""
Generated by ArcGIS ModelBuilder on : 2024-11-28 12:39:14
"""
import arcpy
import os

def FeatureClassGenerator(workspace, wild_card, feature_type, recursive) :
  with arcpy.EnvManager(workspace = workspace):

    dataset_list = [""]
    if recursive:
      datasets = arcpy.ListDatasets()
      dataset_list.extend(datasets)

    for dataset in dataset_list:
      featureclasses = arcpy.ListFeatureClasses(wild_card, feature_type, dataset)
      for fc in featureclasses:
        yield os.path.join(workspace, dataset, fc), fc


def Model2():  # Model2

    # To allow overwriting outputs change overwriteOutput option to True.
    arcpy.env.overwriteOutput = False

    GrunnkartArealregnskap_FGDB_format = "C:\\Users\\endofs\\Documents\\Geonorge-nedlasting\\GrunnkartArealregnskap FGDB-format"

    for FeatureClass, Name_2_ in FeatureClassGenerator(GrunnkartArealregnskap_FGDB_format, "", "POLYGON", "RECURSIVE"):

        # Process: Repair Geometry (Repair Geometry) (management)
        Repaired_Input_Features = arcpy.management.RepairGeometry(in_features=FeatureClass)[0]

        # Process: Pairwise Dissolve (Pairwise Dissolve) (analysis)
        Output_Feature_Class = "C:\\Users\\endofs\\Documents\\ArcGIS\\Projects\\Standard\\Standard.gdb\\FeatureClass_PairwiseDissolv"
        with arcpy.EnvManager(configKeyword=Name_2_):
            arcpy.analysis.PairwiseDissolve(in_features=Repaired_Input_Features, out_feature_class=Output_Feature_Class)

if __name__ == '__main__':
    # Global Environment settings
    with arcpy.EnvManager(scratchWorkspace="C:\\Users\\endofs\\Documents\\ArcGIS\\Projects\\Standard\\Standard.gdb", workspace="C:\\Users\\endofs\\Documents\\ArcGIS\\Projects\\Standard\\Standard.gdb"):
        Model2()



In [23]:
#Required only when there are SubModel(s)
import sys 
sys.path.append(r"C:\Users\endofs\AppData\Local\Temp\ArcGISProTemp4060")
# -*- coding: utf-8 -*-
"""
Generated by ArcGIS ModelBuilder on : 2024-11-28 12:39:14
"""
import arcpy
import os
import geopandas as gpd
import requests
import pandas as pd

In [27]:
import requests
import geopandas as gpd

def get_total_records(url, params):    
    params.update({
        "returnCountOnly": "true",
        "f": "json"
    })
    
    response = requests.get(url, params=params)
    data = response.json()
    
    return data['count']

def fetch_and_store_data(name, url, params, total_records, max_records=5000):
    all_features = []
    offset = 0
    print(f"Total number of records: {total_records}")
    
    while offset < total_records:
        params.update({
            "resultOffset": offset,
            "returnCountOnly": "false",
            "resultRecordCount": max_records,
            "returnGeometry": "true",
            "f": "geojson"
        })
        
        retries = 0
        success = False
        while retries < 5 and not success:
            response = requests.get(url, params=params)
            if response.status_code == 200:
                try:
                    data = response.json()
                    success = True
                except ValueError:
                    retries += 1
                    print(f"Retry {retries}/5: Failed to parse JSON from URL: {url}")
            else:
                retries += 1
                print(f"Retry {retries}/5: Failed to fetch data from URL: {url}, Status Code: {response.status_code}")
        
        if not success:
            print(f"Failed to fetch data from URL: {url} after 5 retries")
            break

        if 'features' in data:
            print(f"Fetched {len(data['features'])} features from offset {offset}")
            all_features.extend(data['features'])
        else:
            print(f"No features found in response at offset {offset}")
            break
        
        if len(data['features']) < max_records:
            break

        offset += max_records

    return all_features

# COMMAND ----------

urls = {
    "url_artnasjonal_punkt": "https://arcgis06.miljodirektoratet.no/arcgis/rest/services/faktaark/artnasjonal2/MapServer/0/query",
    "url_artnasjonal_omr": "https://arcgis06.miljodirektoratet.no/arcgis/rest/services/faktaark/artnasjonal2/MapServer/1/query"
}


In [28]:
# Initialize an empty list to store GeoDataFrames
gdfs = []

for key, url in urls.items():
    params = {
        "where": "1=1",
        "outFields": "*",
        "returnGeometry": "false",
        "f": "json",
    }
    name = key.replace("url_", "")
    tot_rec = get_total_records(url=url, params=params)
    data = fetch_and_store_data(name=name, url=url, params=params, total_records=tot_rec)
    gdf = gpd.GeoDataFrame.from_features(data)
    
    # Convert geometry to WKB
    if 'geometry' in gdf.columns:
        gdf['geometry_wkb'] = gdf['geometry'].apply(lambda geom: geom.wkb if geom else None)
    
    gdfs.append(gdf)

# Combine all GeoDataFrames into one
combined_gdf = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True))

print(combined_gdf.head())

Total number of records: 2115704
Fetched 5000 features from offset 0
Fetched 5000 features from offset 5000
Fetched 5000 features from offset 10000
Fetched 5000 features from offset 15000
Fetched 5000 features from offset 20000
Fetched 5000 features from offset 25000
Fetched 5000 features from offset 30000
Fetched 5000 features from offset 35000
Fetched 5000 features from offset 40000
Fetched 5000 features from offset 45000
Fetched 5000 features from offset 50000
Fetched 5000 features from offset 55000
Fetched 5000 features from offset 60000
Fetched 5000 features from offset 65000
Fetched 5000 features from offset 70000
Fetched 5000 features from offset 75000
Fetched 5000 features from offset 80000
Fetched 5000 features from offset 85000
Fetched 5000 features from offset 90000
Fetched 5000 features from offset 95000
Fetched 5000 features from offset 100000
Fetched 5000 features from offset 105000
Fetched 5000 features from offset 110000
Fetched 5000 features from offset 115000
Fetched 

In [32]:
combined_gdf.to_file("c:/data/artnasjonalv2.gpkg", driver='GPKG', layer='name')

ValueError: Invalid field type <class 'bytes'>

In [12]:
fetch_and_store_data(name=name,url=urls["url_artnasjonal_punkt"], params=params, total_records=10)

Total number of records: 10
Fetched 1000 features from offset 0


[]

In [37]:
anf3_omr = gpd.read_file("c:/data/artnasjonalv3_dump.gdb", layer = "artnasjonalv3_omr")
anf2_omr = gpd.read_file("c:/data/ArtNasjonal_0000_norge_25833_FILEGDB.gdb", layer = "ArtForvaltningsinteresse_omr")





In [47]:
anf3_omr.head()

,aktivitetsamlet,antallobservasjoner,artnasjonalid,artsgruppe,crc,datofra,datotil,faktaark,framiljodir,fylke,...,omraade,presisjon,roedlistekategori,utvalgskriteriersamlet,verdikategori,vitenskapelignavn,vitenskapelignavnid,Shape_Length,Shape_Area,geometry
0,Næringssøkende,1,,Fugler,9754A17C60AB3025,2013-04-30 00:00:00+00:00,2013-04-30 00:00:00+00:00,https://faktaark.naturbase.no/artnasjonal?id=7...,0,Agder,...,1,341,Datamangel (DD),Spesiell økologisk form,Stor verdi,Motacilla flava flava,4342,1624.078638,96193.5,"MULTIPOLYGON (((52031 6456181, 52063 6456110, ..."
1,"Mulig reproduksjon, Reproduksjon",2,,Fugler,1E58A8658ED19A4F,2011-06-07 00:00:00+00:00,2011-06-18 00:00:00+00:00,https://faktaark.naturbase.no/artnasjonal?id=7...,0,Agder,...,1,10,Datamangel (DD),Spesiell økologisk form,Stor verdi,Motacilla flava flava,4342,2881.060589,357848.5,"MULTIPOLYGON (((57686 6456787, 57717 6456697, ..."
2,"Mulig reproduksjon, Næringssøkende",19,,Fugler,D4B7D62FA3E9B40E,2009-04-29 00:00:00+00:00,2024-04-29 00:00:00+00:00,https://faktaark.naturbase.no/artnasjonal?id=7...,0,Agder,...,1,10,Datamangel (DD),Spesiell økologisk form,Stor verdi,Motacilla flava flava,4342,3272.792222,503084.0,"MULTIPOLYGON (((57757 6456700, 58001 6456913, ..."
3,"Mulig reproduksjon, Næringssøkende",5,,Fugler,D4B7D62F422D3329,2017-05-22 00:00:00+00:00,2019-05-21 00:00:00+00:00,https://faktaark.naturbase.no/artnasjonal?id=7...,0,Agder,...,1,10,Datamangel (DD),Spesiell økologisk form,Stor verdi,Motacilla flava flavissima,4343,3272.792222,503084.0,"MULTIPOLYGON (((57757 6456700, 58001 6456913, ..."
4,Næringssøkende,1,,Fugler,C99C95B29F232F3A,2019-04-26 00:00:00+00:00,2019-04-26 00:00:00+00:00,https://faktaark.naturbase.no/artnasjonal?id=7...,0,Agder,...,1,10,Datamangel (DD),Spesiell økologisk form,Stor verdi,Motacilla flava flava,4342,2842.143208,285084.5,"MULTIPOLYGON (((57604 6457107, 57581 6457179, ..."


In [67]:
anf3_omr[anf3_omr["vitenskapelignavnid"]==3681].nunique()

aktivitetsamlet                     100
antallobservasjoner                 120
artnasjonalid                         1
artsgruppe                            1
crc                                3289
datofra                            1700
datotil                            1304
faktaark                           3289
framiljodir                           2
fylke                                21
globalid                           3289
institutioncodesamlet                 2
kommnr                              302
kommune                             302
kr_andrespesielthensynskrevende       1
kr_dd                                 1
kr_fredete                            1
kr_hensynskrevende                    1
kr_kritisktrua                        1
kr_nt                                 1
kr_prioriterte                        1
kr_saarbar                            1
kr_spesielleoekologiskeformer         1
kr_sterkttrua                         1
latitude                           3286


In [68]:
s_anf2_omr = anf2_omr.groupby('vitenskapeligNavnId')['geometry'].nunique().reset_index()
s_anf2_omr.columns = ['vitenskapeligNavnId', 'unique_values_in_geometry']
s_anf2_omr.head()

s_anf3_omr = anf3_omr.groupby('vitenskapelignavnid')['artnasjonalid'].nunique().reset_index()
s_anf3_omr.columns = ['vitenskapelignavnid', 'unique_values_in_artnasjonalid']

s_anf2_omr['vitenskapeligNavnId'] = s_anf2_omr['vitenskapeligNavnId'].astype(str)
s_anf3_omr['vitenskapelignavnid'] = s_anf3_omr['vitenskapelignavnid'].astype(str)

s_anf3_omr = s_anf3_omr.rename(columns={"unique_values_in_geometry": "unikeFunnV3"})
s_anf2_omr = s_anf2_omr.rename(columns={"unique_values_in_geometry": "unikeFunnV2"})

s_anf_omr = s_anf2_omr.merge(s_anf3_omr, how = "outer", left_on = "vitenskapeligNavnId", right_on = "vitenskapelignavnid")
s_anf_omr["Differanse"]=s_anf_omr["unikeFunnV3"]-s_anf_omr["unikeFunnV2"]
s_anf_omr = s_anf_omr.sort_values(by="Differanse", ascending=True)
s_anf_omr.head()
#s_anf_omr.to_csv("c:/data/anf_omr_forskjeller.csv")

Exception ignored in: <function Collection.__del__ at 0x000001F6DF214680>
Traceback (most recent call last):
  File "c:\Users\endofs\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\Lib\site-packages\fiona\collection.py", line 702, in __del__
    def __del__(self):

KeyboardInterrupt: 


KeyError: 'unikeFunnV2'

In [ ]:
anf2_pkt = gpd.read_file("c:/data/ArtNasjonal_0000_norge_25833_FILEGDB.gdb", layer = "ArtForvaltningsinteresse_punkt")
anf3_pkt = gpd.read_file("c:/data/artnasjonalv3_dump.gdb", layer = "artnasjonalv3_pkt")

In [ ]:
anf2

,identifikasjon_lokalId,identifikasjon_navnerom,aktivitet,antallObservasjoner,artsgruppe,datoFra,datoTil,faktaark,forvaltningskategori,forvaltningskategori_beskrivelse,...,krit_spesieltHensynskrevendeArt,krit_truetArt,krit_nærTruetArt,krit_spesiellØkologiskForm,vitenskapeligNavn,vitenskapeligNavnId,listeStatus,listeStatus_beskrivelse,kommune,geometry
0,7.43717_57.989599_3681,no.miljodirektoratet.artnasjonal,mulig reproduksjon,3,fugl,20180608000000,20220426000000,https://faktaark.naturbase.no/artnasjonal/?id=...,1,Arter av særlig stor forvaltningsinteresse,...,None,ja,None,None,Larus canus,3681,VU,Sårbar,4205,"MULTIPOLYGON (((53329.762 6452798.011, 53686.7..."
1,9.039666_63.239948_3681,no.miljodirektoratet.artnasjonal,"mulig reproduksjon,stasjonær",4,fugl,20150609000000,20230514000000,https://faktaark.naturbase.no/artnasjonal/?id=...,1,Arter av særlig stor forvaltningsinteresse,...,None,ja,None,None,Larus canus,3681,VU,Sårbar,5055,"MULTIPOLYGON (((200915.96 7026451.999, 201230...."
2,6.748998_58.060027_3799,no.miljodirektoratet.artnasjonal,"mulig reproduksjon,næringssøkende,reproduksjon...",36,fugl,19760718000000,20230520000000,https://faktaark.naturbase.no/artnasjonal/?id=...,2,Arter av stor forvaltningsinteresse,...,None,None,ja,None,Tringa totanus,3799,NT,Nær truet,4206,"MULTIPOLYGON (((13408.554 6466149.021, 13707.5..."
3,6.555088_58.114846_3518,no.miljodirektoratet.artnasjonal,"næringssøkende,stasjonær",86,fugl,20081019000000,20190509000000,https://faktaark.naturbase.no/artnasjonal/?id=...,1,Arter av særlig stor forvaltningsinteresse,...,None,ja,None,None,Melanitta nigra,3518,VU,Sårbar,4206,"MULTIPOLYGON (((2636.475 6472629.017, 2621.475..."
4,10.844857_59.094048_185920,no.miljodirektoratet.artnasjonal,næringssøkende,1,fugl,20220415000000,20220415000000,https://faktaark.naturbase.no/artnasjonal/?id=...,1,Arter av særlig stor forvaltningsinteresse,...,None,ja,None,None,Chloris chloris,185920,VU,Sårbar,3110,"MULTIPOLYGON (((261997 6557918, 261994 6557931..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98567,10.991229_65.025016_3681,no.miljodirektoratet.artnasjonal,mulig reproduksjon,2,fugl,20220526000000,20240617000000,https://faktaark.naturbase.no/artnasjonal/?id=...,1,Arter av særlig stor forvaltningsinteresse,...,None,ja,None,None,Larus canus,3681,VU,Sårbar,5060,"MULTIPOLYGON (((310638 7216971, 311474 7217679..."
98568,10.991229_65.025016_3694,no.miljodirektoratet.artnasjonal,"mulig reproduksjon,reproduksjon",2,fugl,20220526000000,20240617000000,https://faktaark.naturbase.no/artnasjonal/?id=...,4,Ansvarsarter,...,None,None,None,None,Larus marinus,3694,LC,Livskraftig,5060,"MULTIPOLYGON (((310638 7216971, 311474 7217679..."
98569,5.321629_60.026746_5074,no.miljodirektoratet.artnasjonal,,1,øvrigeDyr,20201001000000,20201001000000,https://faktaark.naturbase.no/artnasjonal/?id=...,2,Arter av stor forvaltningsinteresse,...,None,None,ja,None,Isidella lofotensis,5074,NT,Nær truet,"4616,4625","MULTIPOLYGON (((-39048.351 6695699.059, -36931..."
98570,5.350474_60.351475_4252,no.miljodirektoratet.artnasjonal,"næringssøkende,stasjonær",2,fugl,20240414000000,20241123000000,https://faktaark.naturbase.no/artnasjonal/?id=...,4,Ansvarsarter,...,None,None,None,None,Fringilla montifringilla,4252,LC,Livskraftig,4601,"MULTIPOLYGON (((-31301.302 6729567.062, -31298..."
